# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
import os
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"

In [4]:
# from secrets1 import IEX_CLOUD_API_TOKEN='pk_18e6883dc01b4da48916b1d3fb70d51e'
# print(IEX_CLOUD_API_TOKEN)
IEX_CLOUD_API_TOKEN='pk_18e6883dc01b4da48916b1d3fb70d51e'
print(IEX_CLOUD_API_TOKEN)

pk_18e6883dc01b4da48916b1d3fb70d51e


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol='AAPL'
api_url=f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 58463878, 'calculationPrice': 'tops', 'change': 0.32, 'changePercent': 0.00171, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 188.6, 'iexAskSize': 195, 'iexBidPrice': 187.98, 'iexBidSize': 200, 'iexClose': 187.97, 'iexCloseTime': 1693499743221, 'iexLastUpdated': 1693499743221, 'iexMarketPercent': 0.017400255514299925, 'iexOpen': 187.85, 'iexOpenTime': 1693488600131, 'iexRealtimePrice': 187.97, 'iexRealtimeSize': 15, 'iexVolume': 337226, 'lastTradeTime': 1693499743221, 'latestPrice': 187.97, 'latestSource': 'IEX real time price', 'latestTime': '12:35:43 PM', 'latestUpdate': 1693499743221, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2938766589040, 'oddL

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price=data['latestPrice']
market_cap=data['marketCap']
print(market_cap)

2938766589040


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_col=['Ticker','Stock Price','Market Capitilisation','Number of shares to buy']
final_dataframe=pd.DataFrame(columns=my_col)
final_dataframe
#panda series are 1-D
#panda frames are 2d


,Ticker,Stock Price,Market Capitilisation,Number of shares to buy


In [8]:
final_dataframe._append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index=my_col
    ),
    ignore_index=True    #this is only to have null values
)
    

,Ticker,Stock Price,Market Capitilisation,Number of shares to buy
0,AAPL,187.97,2938766589040,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
final_dataframe = pd.DataFrame(columns = my_col)
for symbol in stocks['Ticker'][:5]:
    api_url=f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe._append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_col), 
                                        ignore_index = True)


In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitilisation,Number of shares to buy
0,A,121.465,35877823733,N/A
1,AAL,14.765,9646896368,N/A
2,AAP,69.350,4124262600,N/A
3,AAPL,187.980,2938922931360,N/A
4,ABBV,148.370,261879975912,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(lst,n):
    for i in range (0,len(lst),n):
        yield lst[i:i+n]

In [12]:
symbol_grps=list(chunks(stocks['Ticker'],100))
#here , stocks is dataframe but stocks['ticker'] is series 
#so basically here we dividing the ig list into lists of 100 items
symbol_strings=[]
for i in range(0,len(symbol_grps)):
    symbol_strings.append(','.join(symbol_grps[i]))
    # print(symbol_strings[i])
    
final_dataframe=pd.DataFrame(columns=my_col)
for symbol_string in symbol_strings[:i]:
    batch_api_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            final_dataframe =  final_dataframe._append(
            pd.Series(
                [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
                ],
            index=my_col
            ),
            ignore_index=True
    )
final_dataframe

,Ticker,Stock Price,Market Capitilisation,Number of shares to buy
0,A,121.465,35877823733,N/A
1,AAL,14.760,9643629555,N/A
2,AAP,69.350,4124262600,N/A
3,AAPL,187.950,2938453904400,N/A
4,ABBV,148.370,261879975912,N/A
...,...,...,...,...
491,XLNX,194.920,48414620999,N/A
492,XOM,110.810,443593792727,N/A
493,XRAY,37.250,7886421224,N/A
494,XRX,16.030,2518634049,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [14]:
portfolio_size=input("enter the value:")
try:
    val=float(portfolio_size)
except ValueError:
    print("Thats not a number")
    portfolio_size=input("enter the value:")
    val=float(portfolio_size)

enter the value: 1000000


In [18]:
position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitilisation,Number of shares to buy
0,A,121.465,35877823733,16
1,AAL,14.760,9643629555,136
2,AAP,69.350,4124262600,29
3,AAPL,187.950,2938453904400,10
4,ABBV,148.370,261879975912,13
...,...,...,...,...
491,XLNX,194.920,48414620999,10
492,XOM,110.810,443593792727,18
493,XRAY,37.250,7886421224,54
494,XRX,16.030,2518634049,125


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [26]:
writer=pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [27]:
background_color='#0a0a23'
font_color='#ffffff'

string_format=writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color':background_color,
        'border':1
    }
)
dollar_format=writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color':background_color,
        'border':1
    }
)
integer_format=writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color':background_color,
        'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [34]:
writer.sheets['Recommended Trades'].set_column('A:A',20,string_format)
writer.sheets['Recommended Trades'].set_column('B:B',20,string_format)
writer.sheets['Recommended Trades'].set_column('C:C',20,string_format)
writer.sheets['Recommended Trades'].set_column('D:D',20,string_format)
writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitilisation',dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of shares to buy',integer_format)
writer.close()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy:

AttributeError: 'XlsxWriter' object has no attribute 'save'